In [18]:
from tradingenv import TradingEnv
from tradingenv.contracts import ETF
from tradingenv.spaces import BoxPortfolio
from tradingenv.state import IState
from tradingenv.rewards import RewardLogReturn
from tradingenv.broker.fees import BrokerFees
import yfinance

In [35]:
# Load prices of SPY ETF from Yahoo Finance.
prices = yfinance.Ticker("SPY").history(period="12mo")['Close'].tz_localize(None).to_frame(ETF('SPY'))

In [39]:
# Instance the trading environment.
env = TradingEnv(
    action_space=BoxPortfolio(prices.columns, low=-1, high=+1, as_weights=True),
    state=IState(),
    reward=RewardLogReturn(),
    prices=prices,
    initial_cash=1_000_000,
    latency=0,                # seconds
    steps_delay=1,            # trades are implemented with a delay on one step
    broker_fees=BrokerFees(
        markup=0.005,         # 0.5% broker makup on deposit rate
        proportional=0.0001,  # 0.01% fee of traded notional
        fixed=1,              # $1 per trade
    ),
)

In [43]:
# OpenAI/gym protocol.
obs = env.reset()
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

In [47]:
info

{'_rebalancing': Rebalancing(2023-09-06 00:00:00)}